In [1]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By


def get_user_tweets(user):

    url = f'https://twitter.com/{user}'

    # Define the options for the Chrome webdriver
    options = Options()
    options.add_argument('--headless')
    options.add_argument("--incognito")
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument("--enable-javascript")

    driver = webdriver.Chrome(options=options)

    # Load the page
    driver.get(url)

    # Wait for tweets to populate the page
    try:
        WebDriverWait(driver, 5).until(expected_conditions.presence_of_element_located(
            (By.CSS_SELECTOR, '[data-testid="tweet"]')))
    except WebDriverException:
        return

    # Extract the HTML content
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    posts = soup.find_all(attrs={"data-testid": "tweetText"})

    # Extract the text content of each post
    post_content = [post.text for post in posts]

    # Save the data in a CSV file
    data = pd.DataFrame(post_content)

    driver.quit()

    return(data)

def get_tweets():
    df = pd.read_csv('./data/users.csv', delimiter=',')

    tweets = pd.DataFrame()

    for i in range(0, len(df)):
        if i % 10 == 0:
            print(i,'/',len(df)) 

        tweets = get_user_tweets(df['Twitter_username'][i])

        try:
            tweets.to_csv('./data/twitter_posts.csv', mode='a',index=False, header=False)
        except:
            continue

In [3]:
text = pd.read_csv('./data//twitter_posts.csv', delimiter=',', header=None)
text

,0
0,"Ngoc Pham, a 83 yr old Vietnamese man, was one..."
1,I’m making the following announcement and form...
2,“The most aggressive field operation in Califo...
3,Thank you Supervisor @AaronPeskin for endorsin...
4,#tbt sharing a meal w/my friend @narendramodi ...
...,...
1823,Unborn children should be welcomed in life and...
1824,I will not vote for John Boehner. My statemen...
1825,I believe that warriors like Chris Kyle are so...
1826,Millions of Americans stand with #Israel. RT ...


In [4]:
print(get_user_tweets('realdonaldtrump'))

                                                    0
0   Tonight, @FLOTUS and I tested positive for COV...
1      Going welI, I think! Thank you to all. LOVE!!!
2                      I WON THIS ELECTION, BY A LOT!
3                                        WE WILL WIN!
4   71,000,000 Legal Votes. The most EVER for a si...
5   WE ARE LOOKING REALLY GOOD ALL OVER THE COUNTR...
6    I will be making a statement tonight. A big WIN!
7   I am asking for everyone at the U.S. Capitol t...
8   A$AP Rocky released from prison and on his way...
9                                    MERRY CHRISTMAS!
10  The United States of America will be designati...
